In [1]:
import vegetables

/home/smgr/.pyenv/versions/co4/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
tomato = vegetables.Tomato('t1', 5)

# test a register collation action
tomato.collate('ripe')

{'age': 5}

In [3]:
vegetables.vegetable_mapper.collect(tomato, ['ripe'])

['9ca6772e-6621-4511-a4a6-ad451a1da91f',
 '2a91b423-4e08-491c-b1d2-5ec25259191e',
 '4a9edb2b-4ac5-467e-82ef-b254829ac2a2']

In [4]:
vegetables.vegetable_mapper.compose(tomato, action_groups=['aging', 'cooking'])

<Component (SQLTable)> vegetable+tomato+tomato_aging_states+tomato_cooking_states

In [5]:
list(vegetables.vegetable_mapper.compose(tomato, action_groups=['aging']).obj.columns)

[Column('id', Integer(), table=<vegetable>, primary_key=True, nullable=False),
 Column('name', String(), table=<vegetable>),
 Column('color', String(), table=<vegetable>),
 Column('id', Integer(), table=<tomato>, primary_key=True, nullable=False),
 Column('name', String(), ForeignKey('vegetable.name'), table=<tomato>),
 Column('radius', Integer(), table=<tomato>),
 Column('id', Integer(), table=<tomato_aging_states>, primary_key=True, nullable=False),
 Column('name', String(), ForeignKey('tomato.name'), table=<tomato_aging_states>),
 Column('state', String(), table=<tomato_aging_states>),
 Column('age', Integer(), table=<tomato_aging_states>)]

In [6]:
tomato.__class__.__mro__

(vegetables.Tomato, vegetables.Vegetable, co3.co3.CO3, object)

In [7]:
from co3.databases import SQLDatabase

db = SQLDatabase('sqlite://') #, echo=True)

In [8]:
db.recreate(vegetables.vegetable_schema)

In [9]:
vegetables.vegetable_mapper.collector.inserts

{<Component (SQLTable)> vegetable: [{'name': 't1', 'color': 'red'}],
 <Component (SQLTable)> tomato: [{'name': 't1', 'radius': 5}],
 <Component (SQLTable)> tomato_aging_states: [{'name': 't1',
   'state': 'ripe',
   'age': 2}]}

In [10]:
with db.engine.connect() as connection:
    db.manager.insert_many(
        connection, 
        vegetables.vegetable_mapper.collector.inserts,
    )

In [11]:
with db.engine.connect() as connection:
    print(db.accessor.select(
        connection, 
        vegetables.vegetable_schema.get_component('tomato')
    ))

[{'id': 1, 'name': 't1', 'radius': 5}]


In [32]:
agg_table = vegetables.vegetable_mapper.compose(tomato)

with db.engine.connect() as connection:
    agg_res = db.accessor.select(
        connection, 
        agg_table,
        mappings=True,
    )

print(agg_res)

[{'id': 1, 'name': 't1', 'color': 'red', 'id_1': 1, 'name_1': 't1', 'radius': 5}]


In [33]:
agg_table = vegetables.vegetable_mapper.compose(tomato, action_groups=['aging'])#, outer=True)

with db.engine.connect() as connection:
    agg_res = db.accessor.select(
        connection, 
        agg_table,
        mappings=True,
    )

print(agg_res)

[{'id': 1, 'name': 't1', 'color': 'red', 'id_1': 1, 'name_1': 't1', 'radius': 5, 'id_2': 1, 'name_2': 't1', 'state': 'ripe', 'age': 2}]


In [31]:
type(agg_res[0])

sqlalchemy.engine.row.RowMapping

In [15]:
import sqlalchemy as sa
from co3.engines import SQLEngine

a = SQLEngine.execute(db.engine.connect(), sa.select(agg_table.obj))

In [19]:
a.mappings().all()


[]

In [37]:
tomato2 = vegetables.Tomato('t2', 8)

In [38]:
vegetables.vegetable_mapper.collect(tomato2)

AttributeError: 'Tomato' object has no attribute 'action_map'

In [ ]:
db.insert(vegetables.vegetable_schema.get_component('tomato'), 

In [36]:
db.select(vegetables.vegetable_schema.get_component('tomato'))

[{'id': 1, 'name': 't1', 'radius': 5}]